Laboratory exercise due to 24th of April 2024. 

Load the necessary libraries:

In [13]:
library(tidyverse)
library(dplyr)
library(geosphere)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


1) read the data and import in a data.frame or tibble structure

In [14]:
df2 <- read.csv("Data_CitiBike/JC-201902-citibike-tripdata.csv")
df3 <- read.csv("Data_CitiBike/JC-201903-citibike-tripdata.csv")
df4 <- read.csv("Data_CitiBike/JC-201904-citibike-tripdata.csv")
df5 <- read.csv("Data_CitiBike/JC-201905-citibike-tripdata.csv")
df6 <- read.csv("Data_CitiBike/JC-201906-citibike-tripdata.csv")

2) merge the five data frames in a unique structure

In [19]:
df <- rbind(df2, df3, df4, df5, df6)

In [20]:
head(df)

,tripduration,starttime,stoptime,start.station.id,start.station.name,start.station.latitude,start.station.longitude,end.station.id,end.station.name,end.station.latitude,end.station.longitude,bikeid,usertype,birth.year,gender
,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<int>,<chr>,<int>,<int>
1,142,2019-02-01 15:35:02.0820,2019-02-01 15:37:24.1360,3183,Exchange Place,40.71625,-74.03346,3639,Harborside,40.71925,-74.03423,29677,Subscriber,1963,1
2,223,2019-02-01 17:00:46.8900,2019-02-01 17:04:30.5500,3183,Exchange Place,40.71625,-74.03346,3681,Grand St,40.71518,-74.03768,26234,Subscriber,1992,2
3,106,2019-02-01 17:08:01.3260,2019-02-01 17:09:47.4400,3183,Exchange Place,40.71625,-74.03346,3184,Paulus Hook,40.71415,-74.03355,29588,Subscriber,1960,1
4,370,2019-02-01 17:09:31.2100,2019-02-01 17:15:41.6550,3183,Exchange Place,40.71625,-74.03346,3211,Newark Ave,40.72153,-74.04630,29250,Subscriber,1976,1
5,315,2019-02-01 17:19:53.2490,2019-02-01 17:25:09.1400,3183,Exchange Place,40.71625,-74.03346,3273,Manila & 1st,40.72165,-74.04288,29586,Subscriber,1980,1
6,145,2019-02-01 17:32:53.2630,2019-02-01 17:35:18.7510,3183,Exchange Place,40.71625,-74.03346,3214,Essex Light Rail,40.71277,-74.03649,26153,Subscriber,1984,1


3) check for missing data and remove it, if any

In [21]:
for 

[1] "    142" "    223" "    106" "    370" "    315" "    145" "   1625"
    [8] "    485" "    632" "    235" "    306" "    763" "    232" "    242"
   [15] "    150" "    432" "    329" "    963" "    284" "    110" "    219"
   [22] "     79" "   1196" "    541" "    821" "   1497" "   1027" "    888"
   [29] "    566" "    554" "    370" "    708" "    714" "    477" "    340"
   [36] "     80" "     89" "    207" "    169" "    164" "   1013" "    354"
   [43] "    582" "    268" "    405" "    544" "    533" "    807" "   1602"
   [50] "    255" "    113" "    241" "    301" "    104" "    595" "    186"
   [57] "    201" "    257" "    508" "    331" "    712" "    520" "    203"
   [64] "    176" "    265" "    433" "    482" "    200" "    277" "    260"
   [71] "    141" "    303" "    250" "   1544" "    177" "    386" "    242"
   [78] "    225" "    252" "     78" "    408" "    268" "    295" "    201"
   [85] "    328" "    676" "    101" "    316" "    286" "    189" "    881"
   [92] "    283" "    865" "    428" "    175" "    282" "    221" "    423"
   [99] "    129" "    324" "    264" "    286" "    609" "    277" "    232"
  [106] "    757" "    252" "    157" "    215" "    352" "    328" "    172"
  [113] "    498" "    106" "    246" "    194" "    304" "    190" "    438"
  [120] "    143" "    104" "    225" "    159" "    294" "     75" "    425"
  [127] "    834" "     74" "    522" "    100" "    296" "    114" "    271"
  [134] "    189" "    391" "    203" "    220" "   1991" "    292" "    314"
  [141] "    131" "    222" "    534" "    673" "    324" "    564" "    111"
  [148] "    246" "    645" "    521" "    380" "    102" "    181" "    167"
  [155] "    235" "     72" "    259" "    884" "    237" "    134" "    574"
  [162] "    137" "    216" "    277" "    130" "    290" "   2032" "    327"
  [169] "    175" "    663" "    230" "    109" "    191" "    137" "    204"
  [176] "    143" "    203" "    210" "    894" "    323" "    227" "    205"
  [183] "    253" "    215" "    691" "    573" "    172" "    326" "    220"
  [190] "    308" "    523" "    105" "    233" "    259" "    514" "    217"
  [197] "    267" "    439" "     82" "    554" "    377" "    802" "    635"
  [204] "    555" "    356" "    411" "     83" "   1269" "    454" "    302"
  [211] "    917" "    274" "    280" "    382" "    358" "    446" "    356"
  [218] "    312" "    218" "    419" "    638" "    558" "    340" "    127"
  [225] "    284" "    136" "    156" "    344" "    384" "    268" "    299"
  [232] "    285" "    213" "    493" "    565" "    216" "    326" "    247"
  [239] "    277" "    253" "    309" "    361" "    611" "    328" "    309"
  [246] "   1017" "    324" "    326" "    118" "    364" "    546" "   1209"
  [253] "    430" "    163" "    267" "    775" "    304" "   1046" "    313"
  [260] "    680" "    617" "    252" "    283" "    292" "    121" "     68"
  [267] "     78" "   1844" "     72" "    275" "    269" " 520186" "    209"
  [274] "    211" "    263" "    110" "    297" "    691" "    239" "    325"
  [281] "    277" "    231" "    148" "    162" "    232" "    273" "    295"
  [288] "    246" "    250" "    467" "    455" "    228" "    130" "    799"
  [295] "    160" "    235" "    215" "    278" "    158" "    298" "    318"
  [302] "    297" "    210" "    302" "    394" "    308" "    361" "    201"
  [309] "    646" "    135" "    123" "    119" "    346" "     73" "    137"
  [316] "    283" "    414" "    255" "    235" "    580" "    350" "    221"
  [323] "    447" "    285" "    298" "    129" "   1303" "    446" "    108"
  [330] "    417" "    223" "    370" "    349" "    417" "    366" "    283"
  [337] "    216" "    337" "    245" "    341" "    383" "    263" "    751"
  [344] "    255" "    323" "    281" "    132" "    330" "    158" "    506"
  [351] "    272" "    360" "    403" "   2730" "    338" "    518" "    481"
  [358] "    255" "    284

In [18]:
head(df)

[1] "    142" "    223" "    106" "    370" "    315" "    145"

4.1) compute the average and the median trip duration in minutes